In [1]:
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import helper
from glob import glob
import os
from PIL import Image
data_dir = './DataSet'
data_files = glob(os.path.join(data_dir, 'Bin*.jpg'))
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
shape = len(data_files), IMAGE_WIDTH, IMAGE_HEIGHT, 3
shape

(50000, 256, 256, 3)

In [3]:
def get_batch(image_files, width, height, mode='RGB'):
    """
    Get a single image
    """
    data_batch = np.array(
        [np.array((Image.open(sample_file)).convert(mode)) for sample_file in image_files]).astype(np.float32)

    # Make sure the images are in 4 dimensions
    if len(data_batch.shape) < 4:
        data_batch = data_batch.reshape(data_batch.shape + (1,))

    return data_batch

def get_batches(batch_size):
    """
    Generate batches
    """
    IMAGE_MAX_VALUE = 255


    current_index = 0
    while current_index + batch_size <= shape[0]:
        data_batch = get_batch(
            data_files[current_index:current_index + batch_size],
            *shape[1:3])

        current_index += batch_size

        yield data_batch / IMAGE_MAX_VALUE - 0.5

In [4]:
def model_inputs(image_width, image_height, image_channel, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, image_height,image_width, image_channel), name = 'input_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name = 'input_z')
    learning_rate = tf.placeholder(tf.float32, name = 'learning_rate')
    
    return inputs_real, inputs_z, learning_rate

In [5]:
def discriminator(images, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        alpha = 0.2
        x1 = tf.layers.conv2d(images, 64, 5, strides = 2, padding = 'same')
        relu1 = tf.maximum(alpha*x1,x1)
        
        x2 = tf.layers.conv2d(relu1, 128, 5, strides = 2, padding = 'same')
        bn2 = tf.layers.batch_normalization(x2, training = True)
        relu2 = tf.maximum(alpha*x2, x2)
        
        x3 = tf.layers.conv2d(relu2, 256, 5, strides = 1, padding = 'same')
        bn3 = tf.layers.batch_normalization(x3, training = False)
        relu3 = tf.maximum(alpha*x3, x3)
        
        flat = tf.reshape(relu3, (-1,4*4*256))
        logits = tf.layers.dense(flat, 1)
        out = tf.sigmoid(logits)
        
        return out,logits
    

In [6]:
def generator(z, out_channel_dim, is_train = True):
    with tf.variable_scope('generator', reuse=False if is_train==True else True):
        
        alpha = 0.2
        
        x1 = tf.layers.dense(z, 2*2*512)
        x1 = tf.reshape(x1, (-1, 2, 2, 512))
        x1 = tf.layers.batch_normalization(x1, training = is_train)
        x1 = tf.maximum(alpha*x1,x1)
        
        x2 = tf.layers.conv2d_transpose(x1, 256, 5, strides = 2, padding = 'VALID')
        x2 = tf.layers.batch_normalization(x2, training = is_train)
        x2 = tf.maximum(alpha*x2, x2)
        
        x3 = tf.layers.conv2d_transpose(x2, 128, 5, strides = 2, padding = 'same')
        x3 = tf.layers.batch_normalization(x3, training = is_train)
        x3 = tf.maximum(alpha*x3, x3)
        
        logits = tf.layers.conv2d_transpose(x3, out_channel_dim, 5, strides = 2, padding = 'same')
        
        out = tf.tanh(logits)
        
        return out
        

In [7]:
def model_loss(input_real, input_z, out_channel_dim):
    label_smoothing = 0.9
    
    g_model = generator(input_z, out_channel_dim)
    d_model_real, d_logits_real = discriminator(input_real)
    d_model_fake, d_logits_fake = discriminator(g_model, reuse=True)
    
    d_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real,
                                                labels=tf.ones_like(d_model_real) * label_smoothing))
    d_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                labels=tf.zeros_like(d_model_fake)))
    
    d_loss = d_loss_real + d_loss_fake
                                                  
    g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                labels=tf.ones_like(d_model_fake) * label_smoothing))
    
    
    return d_loss, g_loss

In [8]:
def model_opt(d_loss, g_loss, learning_rate, beta1):
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): 
        
        d_train_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)

        return d_train_opt, g_train_opt


In [9]:
import numpy as np
import helper
def show_generator_output(sess, n_images, input_z, out_channel_dim):
    z_dim = input_z.get_shape().as_list()[-1]
    example_z = np.random.uniform(-1, 1, size=[n_images, z_dim])

    samples = sess.run(
        generator(input_z, out_channel_dim, False),
        feed_dict={input_z: example_z})

    
    plt.imshow(samples.squeeze())
    plt.show()

In [10]:
def train(epoch_count, batch_size, z_dim, learning_rate, beta1, get_batches, data_shape):
    """
    Train the GAN
    """
    input_real, input_z, _ = model_inputs(data_shape[1], data_shape[2], data_shape[3], z_dim)
    d_loss, g_loss = model_loss(input_real, input_z, data_shape[3])
    d_opt, g_opt = model_opt(d_loss, g_loss, learning_rate, beta1)
    
    steps = 0
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch_i in range(epoch_count):
            for batch_images in get_batches(batch_size):
                
                # values range from -0.5 to 0.5, therefore scale to range -1, 1
                batch_images = batch_images * 2
                steps += 1
            
                batch_z = np.random.uniform(-1, 1, size=(batch_size, z_dim))
                
                _ = sess.run(d_opt, feed_dict={input_real: batch_images, input_z: batch_z})
                _ = sess.run(g_opt, feed_dict={input_real: batch_images, input_z: batch_z})
                
                if steps % 400 == 0:
                    # At the end of every 10 epochs, get the losses and print them out
                    train_loss_d = d_loss.eval({input_z: batch_z, input_real: batch_images})
                    train_loss_g = g_loss.eval({input_z: batch_z})

                    print("Epoch {}/{}...".format(epoch_i+1, epochs),
                          "Discriminator Loss: {:.4f}...".format(train_loss_d),
                          "Generator Loss: {:.4f}".format(train_loss_g))
                    
                  #  _ = show_generator_output(sess, 1, input_z, data_shape[3])

In [ ]:
batch_size = 16
z_dim = 100
learning_rate = 0.0002
beta1 = 0.5
epochs = 2

with tf.Graph().as_default():
    train(epochs, batch_size, z_dim, learning_rate, beta1, get_batches, shape)

Epoch 1/2... Discriminator Loss: 0.3721... Generator Loss: 3.8298
Epoch 1/2... Discriminator Loss: 0.3379... Generator Loss: 5.5845
Epoch 1/2... Discriminator Loss: 0.3289... Generator Loss: 6.0023
Epoch 1/2... Discriminator Loss: 0.3336... Generator Loss: 6.8066
Epoch 1/2... Discriminator Loss: 0.3302... Generator Loss: 7.0279
Epoch 1/2... Discriminator Loss: 0.3261... Generator Loss: 7.2199
Epoch 1/2... Discriminator Loss: 0.3270... Generator Loss: 7.3583
Epoch 2/2... Discriminator Loss: 0.3259... Generator Loss: 7.5594
Epoch 2/2... Discriminator Loss: 0.3264... Generator Loss: 7.6886
Epoch 2/2... Discriminator Loss: 0.3269... Generator Loss: 7.8367
Epoch 2/2... Discriminator Loss: 0.3257... Generator Loss: 8.2482
Epoch 2/2... Discriminator Loss: 0.3255... Generator Loss: 8.2696
Epoch 2/2... Discriminator Loss: 0.3254... Generator Loss: 8.2697
Epoch 2/2... Discriminator Loss: 0.3283... Generator Loss: 8.4550


In [ ]:
z_placeholder = tf.placeholder(tf.float32, [None, z_dim])
sample_image = generator(z_placeholder, 3)
z_batch = np.random.normal(-1, 1, size=[1, z_dim])
temp = (sess.run(sample_image, feed_dict={z_placeholder: z_batch}))
my_i = temp.squeeze()
plt.imshow(my_i)
plt.show()